## Loading all data and labels from file

In [185]:
# Loading all data and labels from file
import h5py
import numpy as np
from keras.utils.np_utils import to_categorical

feature_file = "../preprocess/MSRII-c3d-features.h5"
mode = "r"
with h5py.File(feature_file, mode) as h5:
        '''
        print(h5.keys())
        print(h5['features'])
        print(h5['labels'])
        '''
        data = np.array(h5['features'])
        labels = np.array(h5['labels'])
        labels[np.where(labels > 0)] = 1
        labels = to_categorical(labels, 2)

print(data.shape)
print(labels.shape)

# Seperate data and labels into one training set and one test set
data_rows, _ = data.shape
print(data_rows)

np.random.seed(1993)
rand_permed_rows_seed = np.random.permutation(data_rows)
print(rand_permed_rows_seed)

rand_permed_data = data[rand_permed_rows_seed]
rand_permed_labels = labels[rand_permed_rows_seed]

test_data_rows = 500
X_test = rand_permed_data[0:test_data_rows, :]
y_test = rand_permed_labels[0:test_data_rows, :]
X_train = rand_permed_data[test_data_rows: , :]
y_train = rand_permed_labels[test_data_rows: , :]


(2359, 4096)
(2359, 2)
2359
[1160  640 2349 ..., 1969 1721 1441]


## MLP model

In [220]:
# MLP model (with tanh sigmoid layers)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Create model
model = Sequential()

# Dense(#) is a fully-connected layer with [#] hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 4096-dimensional vectors.
model.add(Dense(256, input_dim=4096, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(256, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(256, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(2, init='uniform'))
model.add(Activation('softmax'))


# Compile model
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_80 (Dense)                 (None, 256)           1048832     dense_input_33[0][0]             
____________________________________________________________________________________________________
activation_80 (Activation)       (None, 256)           0           dense_80[0][0]                   
____________________________________________________________________________________________________
dropout_48 (Dropout)             (None, 256)           0           activation_80[0][0]              
____________________________________________________________________________________________________
dense_81 (Dense)                 (None, 256)           65792       dropout_48[0][0]                 
___________________________________________________________________________________________

In [222]:
batch_size = 16
nb_epoch = 1000

best_accuracy = 0.0
best_epoch = 0
np.random.seed(1993)
for ep in range(1, nb_epoch):
    model.fit(X_train,
              y_train,
              batch_size=batch_size,
              nb_epoch=1, verbose=0,
              validation_data=(X_test, y_test))

    loss, accuracy = model.evaluate(X_test,
                                    y_test, verbose=0,
                                    batch_size=batch_size)
    print('epoch\t' + str(ep) + ':\t(loss, accuracy) = ' + str(loss) + ',\t' + str(accuracy))
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_epoch = ep
        best_W = model.get_weights()


print('-------------------------------------------------------------------------------')
print('best_accuracy: ' + str(best_accuracy) + ' generated at epoch ' + str(best_epoch))
        
model.reset_states()
model.set_weights(best_W)


epoch	1:	(loss, accuracy) = 0.251594476402,	0.906
epoch	2:	(loss, accuracy) = 0.292209318966,	0.888
epoch	3:	(loss, accuracy) = 0.261142904997,	0.906
epoch	4:	(loss, accuracy) = 0.230708302677,	0.922
epoch	5:	(loss, accuracy) = 0.251323549628,	0.904
epoch	6:	(loss, accuracy) = 0.252761283785,	0.91
epoch	7:	(loss, accuracy) = 0.240917190671,	0.916
epoch	8:	(loss, accuracy) = 0.259165244006,	0.906
epoch	9:	(loss, accuracy) = 0.244283193827,	0.916
epoch	10:	(loss, accuracy) = 0.243788685799,	0.912
epoch	11:	(loss, accuracy) = 0.257047909826,	0.908
epoch	12:	(loss, accuracy) = 0.23200116241,	0.918
epoch	13:	(loss, accuracy) = 0.246106331408,	0.908
epoch	14:	(loss, accuracy) = 0.243051918,	0.916
epoch	15:	(loss, accuracy) = 0.239687935829,	0.912
epoch	16:	(loss, accuracy) = 0.22238649416,	0.918
epoch	17:	(loss, accuracy) = 0.271733411163,	0.914
epoch	18:	(loss, accuracy) = 0.22561724329,	0.91
epoch	19:	(loss, accuracy) = 0.245777348161,	0.912
epoch	20:	(loss, accuracy) = 0.236858991385,	0.9

In [219]:
# Test
loss, accuracy = model.evaluate(X_test, y_test, batch_size=16)
accuracy


320/500 [==================>...........] - ETA: 0s

0.93600000000000005